# Reviews Company

In [6]:
# import required packages
import pandas as pd
import numpy as np
import spacy
from spacy.matcher import PhraseMatcher

In [5]:
data = pd.read_csv("Data_Merge_Reviews.csv")
data

,Job Title,Job Rating,Time,Job Status,Pros,Cons,Company_Name
0,Brand Ambassador,5.0,2025-08-08,"Former employee, more than 1 year","Flexible Hours, Supportive Team, Good Training...","Inconsistent Hours, Limited Career Growth, Com...",Safaricom
1,Senior Mobile Developer,4.0,2025-07-28,"Current contractor, more than 1 year",Great For Gaining Experience On Large Scale En...,A Lot Of Shifting Priorities,Safaricom
2,Project Manager,4.0,2025-07-15,"Former employee, more than 3 years",Strong Support For Professional Growth Through...,Occasional Tight Deadlines And HighPressure Ti...,Safaricom
3,Ict Technician,5.0,2025-07-18,Former employee,The Working Experience Was Good,There Are No Major Drawbacks I Can Talk About,Safaricom
4,Intern,5.0,2025-07-15,"Former intern, less than 1 year",Great Work Environment. Regular Bonuses.,None I Can Think Of,Safaricom
...,...,...,...,...,...,...,...
25037,Planning officer,4.0,2022-03-31,Former employee,have good medical covers for employees,terms of employment are contract,Equity
25038,Senior business analyst,4.0,2022-05-14,Current employee,Has good Learning potential for staff,Working schedule is not flexible,Equity
25039,Senior program manager,3.0,2016-05-02,Former employee,Potential for great social impact,Can be slow at times,Equity
25040,Ban assurance officer,1.0,2022-10-17,"Former employee, more than 1 year",Medical insurance cover. Low interest loans fo...,Bad management Less than appropriate pay Extre...,Equity


In [9]:
# using 
data2 = data

In [27]:
def split_info(df: pd.DataFrame)-> str:
    df = df.str.split()
    for i in df:
        Employment_type = i[0]
        Role_type = i[1]
        x = f"{i[2]} {i[3]}"
        if i 
    return df
split_info(data2["Job Status"])

more than
more than
more than


IndexError: list index out of range

In [15]:
data2["Job Status"].tolist()

['Former employee, more than 1 year',
 'Current contractor, more than 1 year',
 'Former employee, more than 3 years',
 'Former employee',
 'Former intern, less than 1 year',
 'Current contractor, more than 1 year',
 'Former contractor, more than 3 years',
 'Former intern, less than 1 year',
 'Current employee',
 'Former employee',
 'Current employee, more than 1 year',
 'Former employee',
 'Former employee',
 'Former employee',
 'Current contractor, more than 3 years',
 'Former intern',
 'Current contractor',
 'Current employee, more than 1 year',
 'Current employee, more than 3 years',
 'Current employee, more than 3 years',
 'Former employee',
 'Current contractor, more than 1 year',
 'Former employee, more than 3 years',
 'Former employee, more than 10 years',
 'Former intern, less than 1 year',
 'Former intern, less than 1 year',
 'Current employee, less than 1 year',
 'Current employee, more than 5 years',
 'Former intern, less than 1 year',
 'Current employee, more than 10 years'